# Import Libraries

In [1]:
# plotting
import matplotlib.pyplot as plt

# manipulate data
import pandas as pd

# more plotting
import seaborn as sns

# custom functions
from gtrends.get_daily_data import og_get_daily_trend as get_daily_trend

# pytrends library
from pytrends.dailydata import get_daily_data
from pytrends.request import TrendReq

pytrend = TrendReq()

# make pandas types compatibile with matplotlibs
pd.plotting.register_matplotlib_converters()

sns_dict = {
    "axes.facecolor": "1",
    "grid.color": "0.9",
    "font.family": ["sans-serif"],
    "font.sans-serif": ["Yanone Kaffeesatz", "DejaVu Sans"],
}

sns.set_style(style="whitegrid", rc=sns_dict)
sns.set_palette(palette="deep")
%matplotlib inline

plt.rcParams["figure.figsize"] = [12, 6]
plt.rcParams["figure.dpi"] = 100

# Define Default Values for Data Retrieval

In [2]:
keywords = ["Mario Draghi", "Christine Lagarde", "Euro", "European Central Bank"]

start_date = "2012-01-01"
end_date = "2021-08-31"

# Monthly Data

## Retrieve Monthly Data

In [11]:
monthly_data = (
    pd.DataFrame(index=pd.date_range(start_date, end_date))
    .asfreq("MS")
)

for keyword in keywords:
    
    if keyword == "Mario Draghi":
        timespan = f"{start_date} 2019-10-31" # end of his mandate
    elif keyword == "Christine Lagarde":
        timespan = f"2019-11-01 {end_date}"
    else:
        timespan = f"{start_date} {end_date}"

    print(f"Retrieving monthly data for {keyword}, timespan = {timespan}")
    pytrend.build_payload([keyword], timeframe=timespan)

    downloaded_data = (
        # retrieve the data
        pytrend.interest_over_time()
        # drop unneded col
        .drop(columns="isPartial")
        # cast as integers
        .astype("int")
        # rename columns in snakecase
        .rename(columns={
            keyword: keyword.replace(" ", "_").lower()
        })
    )
    
    monthly_data = monthly_data.merge(downloaded_data, how="left", left_index=True, right_index=True)

Retrieving monthly data for Mario Draghi, timespan = 2012-01-01 2019-10-31
Retrieving monthly data for Christine Lagarde, timespan = 2019-11-01 2021-08-31
Retrieving monthly data for Euro, timespan = 2012-01-01 2021-08-31
Retrieving monthly data for European Central Bank, timespan = 2012-01-01 2021-08-31


In [12]:
monthly_data

,mario_draghi,christine_lagarde,euro,european_central_bank
2012-01-01,26.0,NaN,12,90
2012-02-01,26.0,NaN,12,100
2012-03-01,16.0,NaN,11,78
2012-04-01,24.0,NaN,12,75
2012-05-01,23.0,NaN,18,87
...,...,...,...,...
2021-04-01,NaN,NaN,16,39
2021-05-01,NaN,NaN,17,47
2021-06-01,NaN,NaN,101,31
2021-07-01,NaN,NaN,50,32
